In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# # Import API key
from config import API_key

In [2]:
# import the csv 
weather_data = pd.read_csv("../data/weather_data.csv")
weather_data.head()

,City,Country,Latitude,Longintude,Temperature (F),% Humidity,% Cloud Cover,Wind Speed (MPH)
0,Pochutla,MX,15.74,-96.47,84.99,73,24,3.44
1,Vaini,TO,-21.20,-175.20,77.00,73,75,11.41
2,Hasaki,JP,35.73,140.83,86.77,74,40,4.70
3,Narsaq,GL,60.92,-46.05,46.40,87,100,4.70
4,Hermanus,ZA,-34.42,19.23,55.00,88,99,5.01


In [3]:
# configure gmaps with API key
gmaps.configure(API_key)

In [4]:
# store Latitude and Longitude into locations
locations = weather_data[['Latitude', 'Longintude']].astype(float)


In [5]:
# humidity heatmap all datapoints
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=weather_data['% Humidity'],
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Defining ideal weather conditions:

ideal_df = weather_data[(weather_data['% Humidity'] < 65) &
                        (weather_data['Temperature (F)'] > 60) &
                        (weather_data['Temperature (F)'] < 95) &
                        (weather_data['% Cloud Cover'] > 5) &
                        (weather_data['% Cloud Cover'] < 25) &
                        (weather_data['Wind Speed (MPH)'] > 1) &
                        (weather_data['Wind Speed (MPH)'] < 10)]
ideal_df

,City,Country,Latitude,Longintude,Temperature (F),% Humidity,% Cloud Cover,Wind Speed (MPH)
65,Mandalgovi,MN,45.76,106.27,64.29,59,7,4.94
177,Morro Bay,US,35.37,-120.85,72.97,57,6,5.99
205,Haines Junction,CA,60.75,-137.51,61.30,43,15,7.67
220,Ngunguru,NZ,-35.62,174.50,64.00,3,15,3.00
252,Senador Pompeu,BR,-5.59,-39.37,75.54,56,10,4.90
318,Codajás,BR,-3.84,-62.06,76.75,57,24,1.74
349,Assiut,EG,27.18,31.18,82.40,32,15,3.24
395,Hazorasp,UZ,41.32,61.07,71.60,60,18,2.24
445,Poya,NC,-21.35,165.15,74.73,50,10,4.54
476,Baruun-Urt,MN,46.68,113.28,68.86,62,14,9.35


In [29]:
#create a new dataframe that will add hotels to the ideal locations
hotel_df = ideal_df.copy()

#reset the index
hotel_df.reset_index(drop=True, inplace=True)

hotel_df

,City,Country,Latitude,Longintude,Temperature (F),% Humidity,% Cloud Cover,Wind Speed (MPH)
0,Mandalgovi,MN,45.76,106.27,64.29,59,7,4.94
1,Morro Bay,US,35.37,-120.85,72.97,57,6,5.99
2,Haines Junction,CA,60.75,-137.51,61.30,43,15,7.67
3,Ngunguru,NZ,-35.62,174.50,64.00,3,15,3.00
4,Senador Pompeu,BR,-5.59,-39.37,75.54,56,10,4.90
5,Codajás,BR,-3.84,-62.06,76.75,57,24,1.74
6,Assiut,EG,27.18,31.18,82.40,32,15,3.24
7,Hazorasp,UZ,41.32,61.07,71.60,60,18,2.24
8,Poya,NC,-21.35,165.15,74.73,50,10,4.54
9,Baruun-Urt,MN,46.68,113.28,68.86,62,14,9.35


In [34]:
# Use the Google Places API to find the first hotel for each city 
# used radius of 10000m to ensure all cities have hotels associated.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# search for a hotel for each city, one row at a time
for index, row in hotel_df.iterrows():
    #set the hotel latitude and longitude
    hotelLat = hotel_df.loc[index, 'Latitude']
    hotelLong = hotel_df.loc[index, 'Longintude']
    
#     create the target coordinates
    target_coordinates = (f'{hotelLat}, {hotelLong}')
    
    #set radius to 20000m to ensure a matching hotel can be found
    params = {
        "location": target_coordinates,
        "radius": 30000,
        "type": "lodging",
        "key": API_key
    }
    
    response = requests.get(base_url, params=params)
    places_data = response.json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = places_data['results'][1]['name']
        hotel_df.loc[index, 'Hotel Lat'] = places_data['results'][1]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel Long'] = places_data['results'][1]['geometry']['location']['lng']
        hotelname = places_data['results'][1]['name']
        print(f'Found {hotelname}')

    except:
        print('No hotel found')
        
hotel_df    

Found Temuulen Shop
Found Best Western Plus Royal Oak Hotel
Found Raven's Rest Inn
Found Lupton Lodge
Found Hotel Central
Found Hotel Dona Marlene
Found Assiut hotels Armed Forces
Found Asqar Bussines Hotel
Found Gîte du Cap
Found tansag hotel


,City,Country,Latitude,Longintude,Temperature (F),% Humidity,% Cloud Cover,Wind Speed (MPH),Hotel Name,Hotel Lat,Hotel Long
0,Mandalgovi,MN,45.76,106.27,64.29,59,7,4.94,Temuulen Shop,45.762442,106.273017
1,Morro Bay,US,35.37,-120.85,72.97,57,6,5.99,Best Western Plus Royal Oak Hotel,35.263665,-120.678342
2,Haines Junction,CA,60.75,-137.51,61.30,43,15,7.67,Raven's Rest Inn,60.754037,-137.512471
3,Ngunguru,NZ,-35.62,174.50,64.00,3,15,3.00,Lupton Lodge,-35.658646,174.381469
4,Senador Pompeu,BR,-5.59,-39.37,75.54,56,10,4.90,Hotel Central,-5.590490,-39.374195
5,Codajás,BR,-3.84,-62.06,76.75,57,24,1.74,Hotel Dona Marlene,-3.843021,-62.063349
6,Assiut,EG,27.18,31.18,82.40,32,15,3.24,Assiut hotels Armed Forces,27.202062,31.194197
7,Hazorasp,UZ,41.32,61.07,71.60,60,18,2.24,Asqar Bussines Hotel,41.299483,61.104096
8,Poya,NC,-21.35,165.15,74.73,50,10,4.54,Gîte du Cap,-21.499390,165.259042
9,Baruun-Urt,MN,46.68,113.28,68.86,62,14,9.35,tansag hotel,46.682132,113.281294


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Long"]]

In [37]:
# map hotels over heatmap

marker_locations = hotel_df[['Hotel Lat', 'Hotel Long']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

#Display map
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
 # Add marker layer ontop of heat map
fig = gmaps.figure()


fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))